In [ ]:
import sys
import os
import cv2
import importlib
import torch
import numpy as np
import argparse
import yaml
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageOps
import json
import nltk
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import clip
import kornia
import torchvision
import skimage.color
import random

from utils_func import *
from html_images import *
from sample_func import *
from colorizer import *

In [2]:
# Coco model
ckpt_file = 'C:/MyFiles/CondTran/finals/bert_coco/logs/bert/epoch=144-step=259999.ckpt'
# Imagenet model
#ckpt_file = 'C:/MyFiles/CondTran/finals/bert_final/logs/bert/epoch=14-step=142124.ckpt'

device = 'cuda:0'
colorizer = Colorizer(ckpt_file, device, [256, 256], load_clip=False, load_warper=False)
filltran = colorizer.transformer

c:\Users\lucky\Anaconda3\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
c:\Users\lucky\Anaconda3\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Working with z of shape (1, 1024, 16, 16) = 262144 dimensions.


In [4]:
def sample(x_gray, topk, strokes):
    _, f_gray = filltran.hybrid_vqgan.encode(None, x_gray)
    B = f_gray.shape[0]
    sample_shape = [16, 16]
    rows, cols = f_gray.shape[2:4]

    idx_color = filltran.mask_token * torch.ones([B, rows, cols]).to(device).long()

    if len(strokes) > 0:
        cond = []
        cond_indices = []
        for stk in strokes:
            ind = stk['index']
            color = stk['color']
            color = torch.Tensor(color).to(device)
            color = color / 255.0 * 2.0 - 1.0
            for b in range(B):
                index = torch.Tensor([b, ind[0]//16, ind[1]//16]).long().to(device)
                cond.append(color.unsqueeze(0))
                cond_indices.append(index.unsqueeze(0))
        cond = torch.cat(cond, axis=0)
        cond_indices = torch.cat(cond_indices, axis=0)
    else:
        cond = None
        cond_indices = None

    i = 0
    for r in range(16):
        for c in range(16):
            cond_gray = f_gray.clone()
            cond_gray = cond_gray.reshape(cond_gray.shape[0], cond_gray.shape[1], -1)
            cond_gray = cond_gray.permute(0, 2, 1).contiguous()
            # Input color indices
            logits = filltran.coltran(idx_color, cond_gray, cond, cond_indices)
            logits = logits[:, i, :]
            logits = logits.reshape(-1, logits.shape[-1])
            logits = filltran.top_k_logits(logits, topk)
            probs = F.softmax(logits, dim=-1)
            ix = torch.multinomial(probs, num_samples=1)
            idx_color[:, r, c] = ix[:, 0]
            i += 1

    gen = filltran.hybrid_vqgan.decode(idx_color, f_gray)

    return gen


def getImage(path):
    for ext in ['jpg', 'png', 'JPEG']:
        file = path + '.' + ext
        if os.path.isfile(file):
            return Image.open(file).convert('RGB')
    return None

## Raw visualize

In [ ]:
# Sampling arguments
img_dirs = {'imagenet': 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val', 'coco': 'C:\\MyFiles\\Dataset\\coco\\val2017'}

save_dir = 'C:\\Users\\lucky\\Desktop\\visualize\\raw\\temp2'
topk = 100
num_samples = [5, 5, 5]
img_size = [256, 256]
sample_size = [img_size[0]//16, img_size[1]//16]

html = HTML(save_dir, 'Sample')

#with open('C:\\Users\\lucky\\Desktop\\visualize\\raw\\temp.json', 'rb') as f:
    #visual_files = json.load(f)

visual_files = [{
    "dataset": "imagenet",
    "images": ['']
},
{
    "dataset": "coco",
    "images":['000000467511.jpg']
}]

for dataset in visual_files:
    for f in tqdm(dataset['images']):
        if f.endswith('.jpg') or f.endswith('.png') or f.endswith('.JPEG') or f.endswith('.jpeg'):
            dname = dataset['dataset']
            fname = f.split('.')[0]
            I_color = Image.open(os.path.join(img_dirs[dname], f)).convert('RGB')
            I_gray = I_color.convert('L')
            
            gen_imgs = [I_color, I_gray]

            x_gray = preprocess(I_gray, img_size).to(device)

            comp = getImage(os.path.join('C:\\Users\\lucky\\Desktop\\visualize\\raw\\instcolor', fname))
            gen_imgs.append(comp)
            comp = getImage(os.path.join('C:\\Users\\lucky\\Desktop\\visualize\\raw\\chromagan', fname))
            gen_imgs.append(comp)
            comp = getImage(os.path.join('C:\\Users\\lucky\\Desktop\\visualize\\raw\\coltran', fname))
            gen_imgs.append(comp)

            for n in num_samples:
                x_gray_batch = x_gray.repeat(n, 1, 1, 1)
                gen = sample(x_gray_batch, topk=topk, strokes=[])
                for b in range(gen.shape[0]):
                    img = output_to_pil(gen[b])
                    img = color_resize(I_gray, img)
                    gen_imgs.append(img)
            
            texts = []
            for i in range(np.sum(num_samples)+5):
                texts.append(str(i))
            texts[0] = fname
            texts[1] = 'gray'
            texts[2] = 'instcolor'
            texts[3] = 'chromagan'
            texts[4] = 'coltran'
            
            save_result(html, index=fname, images=gen_imgs, texts=texts)
            html.save()

## Exemplar visualize

In [ ]:
# Sampling arguments
topk = 100
num_samples = [5, 5, 5, 5, 5]
img_size = [256, 256]
sample_size = [img_size[0]//16, img_size[1]//16]
img_path = 'C:\\Users\\lucky\\Desktop\\visualize\\exemplar\\visualize.json'
dataset_path = 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val'
ref_path = 'C:\\MyFiles\\Comparison\\exemplar\\final_exp\\images\\1'
save_dir = 'C:\\Users\\lucky\\Desktop\\visualize\\exemplar\\exp2'

html = HTML(save_dir, 'Sample')

# Load strokes
with open(img_path, 'r') as file:
    all_files = json.load(file)

pbar = tqdm(all_files)

for filename in pbar:
    name = filename.split('.')[0]
    gen_imgs = []

    I_color = Image.open(os.path.join(dataset_path, filename.replace('.png', '.JPEG'))).convert('RGB')
    I_gray = I_color.convert('L')
    I_ref = Image.open(os.path.join(ref_path, filename)).convert('RGB')

    gen_imgs.append(I_color)
    gen_imgs.append(I_ref)

    strokes, warped_img = colorizer.get_strokes_from_exemplar(I_gray, I_ref)
    gen_imgs.append(warped_img.resize(I_color.size))

    draw_img = draw_strokes(I_gray, img_size, strokes)
    gen_imgs.append(draw_img.resize(I_color.size))

    gen_imgs.append(Image.open(f'C:\\MyFiles\\Comparison\\exemplar\\deep_exemplar\\{filename}').convert('RGB'))
    gen_imgs.append(Image.open(f'C:\\MyFiles\\Comparison\\exemplar\\exemplar_video\\{filename}').convert('RGB'))

    x_gray = preprocess(I_gray, img_size).to(device)
    
    for n in num_samples:
        x_gray_batch = x_gray.repeat(n, 1, 1, 1)
        gen = sample(x_gray_batch, topk=topk, strokes=strokes)
        for b in range(gen.shape[0]):
            img = output_to_pil(gen[b])
            img = color_resize(I_gray, img)
            gen_imgs.append(img)
        
    texts = []
    for i in range(np.sum(num_samples)+6):
        texts.append(str(i))
    texts[0] = name

    save_result(html, index=name, images=gen_imgs, texts=texts)
    html.save()

## Stroke Visualize

In [ ]:
# Sampling arguments
img_dirs = {'imagenet': 'C:\\MyFiles\\Comparison\\stroke\\final_stroke_imagenet\\images\\0', 
            'coco': 'C:\\MyFiles\\Comparison\\stroke\\final_stroke_coco\\images\\0'}
with open('C:\\MyFiles\\CondTran\\data\\imagenet_strokes.json', 'rb') as f:
    imagenet_strokes = json.load(f)
with open('C:\\MyFiles\\CondTran\\data\\coco_strokes.json', 'rb') as f:
    coco_strokes = json.load(f)
num_strokes = [2, 16]

imagenet_sample_strokes = []
random.seed(100)
for file in imagenet_strokes:
    strokes = file['strokes']
    n_strokes = random.randint(num_strokes[0], num_strokes[1])
    n_strokes = min(n_strokes, len(strokes))
    strokes = random.sample(strokes, k=n_strokes)
    imagenet_sample_strokes.append(strokes)

coco_sample_strokes = []
random.seed(100)
for file in coco_strokes:
    strokes = file['strokes']
    n_strokes = random.randint(num_strokes[0], num_strokes[1])
    n_strokes = min(n_strokes, len(strokes))
    strokes = random.sample(strokes, k=n_strokes)
    coco_sample_strokes.append(strokes)

strokes = {'imagenet': imagenet_sample_strokes, 'coco': coco_sample_strokes}


save_dir = 'C:\\Users\\lucky\\Desktop\\visualize\\stroke\\ours3'
topk = 200
num_samples = [5, 5, 5,5,5,5,5,5,5,5,5,5]
img_size = [256, 256]
sample_size = [img_size[0]//16, img_size[1]//16]

html = HTML(save_dir, 'Sample')

with open('C:\\Users\\lucky\\Desktop\\visualize\\stroke\\visualize.json', 'rb') as f:
    visual_files = json.load(f)

allfiles = []
for files in visual_files:
    for f in tqdm(files['images']):
        fname = f.split('.')[0]
        dataset = files['dataset']
        I_color = Image.open(os.path.join(img_dirs[dataset], f)).convert('RGB')
        I_gray = I_color.convert('L')
        
        gen_imgs = [I_color, I_gray]

        stk = strokes[dataset][int(fname)]

        draw_img = draw_strokes(I_gray, img_size, stk)
        gen_imgs.append(draw_img)

        x_gray = preprocess(I_gray, img_size).to(device)

        for n in num_samples:
            x_gray_batch = x_gray.repeat(n, 1, 1, 1)
            gen = sample(x_gray_batch, topk=topk, strokes=stk)
            for b in range(gen.shape[0]):
                img = output_to_pil(gen[b])
                img = color_resize(I_gray, img)
                gen_imgs.append(img)
        
        texts = []
        for i in range(np.sum(num_samples)+3):
            texts.append(str(i))
        texts[0] = fname

        save_result(html, index=fname, images=gen_imgs, texts=texts)
        html.save()

## Sample text

In [ ]:
# Sampling arguments
topk = 100
num_samples = [1]
img_size = [256, 256]
sample_size = [img_size[0]//16, img_size[1]//16]
file_path = 'C:\\Users\\lucky\\Desktop\\colorization_results\\text\\visualize.json'
dataset_path = 'C:\\Users\\lucky\\Desktop\\colorization_results\\text\\final_text_old_color\\images\\0'
save_dir = 'C:\\Users\\lucky\\Desktop\\More_results\\text\\ours_coco_text'

html = HTML(save_dir, 'Sample')

with open('C:\\MyFiles\\CondTran\\data\\all_text_strokes.json', 'rb') as f:
    images = json.load(f)

# Load strokes
with open(file_path, 'r') as file:
    all_files = json.load(file)

pbar = tqdm(all_files)

for filename in pbar:
    index = filename.split('.png ')[0]
    name = images[int(index)]['image'].split('.')[0]
    caption = filename.split('.png ')[1]
    gen_imgs = []

    I_color = Image.open(os.path.join(dataset_path, name+'.png')).convert('RGB')
    I_gray = I_color.convert('L')

    gen_imgs.append(I_color)

    strokes, heatmaps = colorizer.get_strokes_from_clip(I_gray, caption)

    if len(heatmaps) > 1:
        heat = []
        for h in heatmaps:
            heat.append(np.array(h))
        heat = np.concatenate(heat, axis=1)
        heat = Image.fromarray(heat)
    else:
        heat = heatmaps[0]
    gen_imgs.append(heat)

    draw_img = draw_strokes(I_gray, img_size, strokes)
    gen_imgs.append(draw_img.resize(I_color.size))

    x_gray = preprocess(I_gray, img_size).to(device)

    for n in num_samples:
        x_gray_batch = x_gray.repeat(n, 1, 1, 1)
        gen = sample(x_gray_batch, topk=topk, strokes=strokes)
        for b in range(gen.shape[0]):
            img = output_to_pil(gen[b])
            img = color_resize(I_gray, img)
            gen_imgs.append(img)
        
    texts = []
    for i in range(np.sum(num_samples)+3):
        texts.append(str(i))
    texts[0] = index
    texts[1] = caption

    save_result(html, index=index, images=gen_imgs, texts=texts)
    html.save()

## Sample old photos

In [5]:
# Sampling arguments
img_dir = 'C:\\Users\\lucky\\Desktop\\test_img\\old_photos'
save_dir = 'C:\\Users\\lucky\\Desktop\\legacy\\uncond'
topk = 100
num_samples = [5, 5, 5, 5]
img_size = [256, 256]
sample_size = [img_size[0]//16, img_size[1]//16]

html = HTML(save_dir, 'Sample')
files = os.listdir(img_dir)

for i, f in enumerate(tqdm(files)):
    if f.endswith('.jpg') or f.endswith('.png') or f.endswith('.JPEG') or f.endswith('.jpeg'):
        fname = f.split('.')[0]
        I_color = Image.open(os.path.join(img_dir, f)).convert('RGB')
        I_gray = I_color.convert('L')
        
        gen_imgs = [I_color, I_gray]

        x_gray = preprocess(I_gray, img_size).to(device)

        for n in num_samples:
            x_gray_batch = x_gray.repeat(n, 1, 1, 1)
            gen = sample(x_gray_batch, topk=topk, strokes=[])
            for b in range(gen.shape[0]):
                img = output_to_pil(gen[b])
                img = color_resize(I_gray, img)
                gen_imgs.append(img)
        
        texts = []
        for n in range(np.sum(num_samples)+2):
            texts.append(str(n))
        texts[0] = str(i)
        texts[1] = fname
        
        save_result(html, index=i, images=gen_imgs, texts=texts)
        html.save()

100%|██████████| 212/212 [9:38:42<00:00, 163.79s/it]  
